In [ ]:
%pip install git+https://github.com/maximilian-heeg/UCell.git

In [ ]:
import scanpy as sc
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from scipy.spatial import KDTree
import scanpy as sc
import matplotlib.pyplot as plt
import numpy as np
import warnings
import os
import seaborn as sns

warnings.filterwarnings("ignore")

In [ ]:
def scatter_with_gaussian_kde(ax, x, y, weights=None, **kwargs):
    from scipy.stats import gaussian_kde

    xy = np.vstack([x, y])

    try:
        z = gaussian_kde(xy, weights=weights)(xy)
    except:
        z = gaussian_kde(xy)(xy)

    ax.scatter(x, y, c=z, **kwargs)

In [ ]:
# import human adata

adata_human = sc.read("../data/adata/human.h5ad")
adata = adata_human[(adata_human.obs.peyers == 0) & (adata_human.obs.CD8_column == 1)]

# Create the mouse signatures

In [ ]:
adata_mouse = sc.read_h5ad("../data/adata/timecourse.h5ad")
adata_mouse = adata_mouse[
    adata_mouse.obs.batch.isin(["day90_SI", "day90_SI_r2"]),
    adata_mouse.var.index.str.upper().isin(adata_human.var.index),
]

adata_mouse.var.index = adata_mouse.var.index.str.upper()

In [ ]:
sc.pp.normalize_total(adata_mouse, target_sum=1e4)
sc.pp.log1p(adata_mouse)

In [ ]:
# Coordinates of the gates
gates_mouse = {
    "Top": {
        "edges": [
            [0.15, 0.5],
            [0.6, 0.7],
            [0.8, 0.7],
            [0.8, 1.03],
            [0.15, 1.03],
        ],
        "label_position": {"x": 0.16, "y": 0.9},
        "fill": "#3A9AB244",
        "stroke": "#3A9AB2",
    },
    "Crypt": {
        "edges": [
            [0.15, 0.48],
            [0.6, 0.68],
            [0.8, 0.68],
            [0.8, 0.25],
            [0.2, 0],
            [0.15, 0],
        ],
        "label_position": {"x": 0.16, "y": 0.05},
        "fill": "#F11B0044",
        "stroke": "#F11B00",
    },
    "Muscularis": {
        "edges": [[0.22, 0], [0.8, 0.23], [6, 0.23], [6, 0], [0.22, 0]],
        "label_position": {"x": 0.6, "y": 0.05},
        "fill": "#BDC88155",
        "stroke": "#BDC881",
    },
}

In [ ]:
def filter_adata_expressed_in_n_cells(adata, percent=0.05):
    bin_Layer = adata.X > 0
    gene_expressed_in_percent_cells = np.mean(bin_Layer, axis=0)
    keep = gene_expressed_in_percent_cells > percent
    adata = adata[:, keep]
    return adata


adata_mouse_cd8 = adata_mouse[adata_mouse.obs.Subtype.isin(["Cd8_T-Cell_P14"])]
adata_mouse_cd8 = filter_adata_expressed_in_n_cells(adata_mouse_cd8)

In [ ]:
# Custom biexponential transformation. Maybe not needed for IF data
def transformation(x, a=0.1, b=0.5, c=1, d=3, f=4, w=1):
    x = np.array(x)
    return a * np.exp(b * ((x - w))) - c * np.exp(-d * (x - w)) + f


def classify_cells(adata, gates, transformation=transformation):
    """
    Classify cells based on the gates.
    """
    from shapely.geometry import Point
    from shapely.geometry.polygon import Polygon
    import geopandas as gpd

    adata.obs["epithelial_distance_transformed"] = transformation(
        adata.obs["epithelial_distance"]
    )
    adata.obs["gate"] = False

    print("Creating polygons")
    polygons = {}
    for gate in gates:
        # Apply transformation to x values
        points = [
            [transformation(element[0])] + element[1:]
            for element in gates[gate]["edges"]
        ]
        polygons[gate] = Polygon(points)
    polygons = gpd.GeoSeries(polygons)
    gpd_poly = gpd.GeoDataFrame({"gates": polygons}, geometry="gates")

    print("Creating cells")
    cells = gpd.GeoSeries.from_xy(
        adata.obs["epithelial_distance_transformed"], adata.obs["crypt_villi_axis"]
    )
    gpd_cells = gpd.GeoDataFrame({"cells": cells}, geometry="cells")

    print("Joining cells and polygons")
    result = gpd.sjoin(
        gpd_cells,
        gpd_poly,
        how="left",
    )
    return result


classification = classify_cells(adata_mouse_cd8, gates_mouse)

adata_mouse_cd8.obs["gate"] = classification["index_right"]

In [ ]:
adata_mouse_cd8 = adata_mouse_cd8[adata_mouse_cd8.obs.gate.isin(["Crypt", "Top"])]

In [ ]:
sc.tl.rank_genes_groups(adata_mouse_cd8, groupby="gate", method="wilcoxon")

In [ ]:
signature_genes = np.array(
    [list(i) for i in adata_mouse_cd8.uns["rank_genes_groups"]["names"]]
)[:15].T

signature_regions = {}
signature_names = ["Crypt", "Top"]
for i in range(len(signature_names)):
    signature_regions[signature_names[i]] = signature_genes[i]

signature_regions

# Project signature to human dataset

In [ ]:
signature_regions_human = {}
signature_names = ["Crypt", "Top"]
for i in range(len(signature_names)):
    signature_regions_human[signature_names[i]] = [
        c.upper() for c in signature_genes[i]
    ]

In [ ]:
import ucell

ucell.add_scores(adata, signatures=signature_regions_human, maxRank=15)

In [ ]:
batches = {
    "human_05_r1": {"x": 2400, "y": 2400},
}

In [ ]:
gates = {
    "Top-IE": {
        "edges": [
            [-0.01, 0.5],
            [0.12, 0.5],
            [0.12, 1.03],
            [-0.01, 1.03],
        ],
        "label_position": {"x": 0.16, "y": 0.9},
        "fill": "#3A9AB244",
    },
    "Top-LP": {
        "edges": [
            [0.13, 0.5],
            [1.2, 0.5],
            [1.2, 1.03],
            [0.13, 1.03],
        ],
        "label_position": {"x": 1, "y": 0.8},
        "fill": "#3A9AB244",
    },
    "Crypt-IE": {
        "edges": [
            [-0.01, 0.48],
            [0.12, 0.48],
            [0.12, 0.13],
            [0.12, 0],
            [-0.01, 0],
        ],
        "label_position": {"x": 0.16, "y": 0.05},
        "fill": "#F11B0044",
    },
    "Crypt-LP": {
        "edges": [
            [0.13, 0.48],
            [1.2, 0.48],
            [1.2, 0],
            [0.13, 0],
        ],
        "label_position": {"x": 1, "y": 0.25},
        "fill": "#F11B0044",
    },
}


def draw_gates(ax, gates, transformation, type="edge"):
    from matplotlib.patches import Polygon

    for gate in gates:
        # Apply transformation to x values
        points = [
            [transformation(element[0])] + element[1:]
            for element in gates[gate]["edges"]
        ]

        if type == "fill":
            p = Polygon(points, facecolor=gates[gate]["fill"], edgecolor="none")
            ax.add_patch(p)
        elif type == "edge":
            p = Polygon(points, facecolor="none", edgecolor="#222222")
            ax.add_patch(p)

In [ ]:
import scipy.stats as stats


# Create subplots
def plot_imaps_signature(
    adata,
    ax_ticks=[0, 0.15, 0.3, 0.6, 1, 6],
    transformation=transformation,
    gates=gates,
    ucell_key="UCell_Crypt",
):
    fig = plt.figure(figsize=(3, 3 * len(batches)), dpi=200)

    # Apply transformation
    adata.obs["epithelial_distance_transformed"] = transformation(
        adata.obs["epithelial_distance_clipped"]
    )

    sub_adata = adata.copy()
    ax = fig.add_subplot(len(batches), 1, 1)

    # Draw gates filled in background
    draw_gates(ax, gates=gates, transformation=transformation, type="fill")

    sns.kdeplot(
        data=sub_adata.obs,
        x="epithelial_distance_transformed",
        y="crypt_villi_axis",
        ax=ax,
        color="#444444",
        linewidths=0.5,
        weights=(
            sub_adata.obs[ucell_key].values - np.min(sub_adata.obs[ucell_key].values)
        )
        ** 2,
        cmap="viridis",
    )

    ## average point value over the 10 nearest points
    points = np.array(
        adata.obs[["epithelial_distance_transformed", "crypt_villi_axis"]]
    )
    # standard scale epithelial_distance transformed
    points[:, 0] = (points[:, 0] - np.min(points[:, 0])) / (
        np.max(points[:, 0]) - (np.min(points[:, 0]))
    )
    tree = KDTree(points)
    distances, indices = tree.query(points, k=50)
    values = adata.obs[ucell_key].values

    average_values = []
    for i in range(len(indices)):
        average_values.append(np.mean(values[indices[i]]))

    # ax.scatter(sub_adata.obs["epithelial_distance_transformed"], sub_adata.obs["crypt_villi_axis"], c=average_values, s=5)
    # Colored scatter plot

    scatter_with_gaussian_kde(
        ax=ax,
        x=sub_adata.obs["epithelial_distance_transformed"],
        y=sub_adata.obs["crypt_villi_axis"],
        s=5,
        weights=(
            sub_adata.obs[ucell_key].values - np.min(sub_adata.obs[ucell_key].values)
        )
        ** 2,
        cmap="viridis",
    )

    # Transform the tick labels and set them
    ax.set_xticks(transformation(ax_ticks))
    ax.set_xticklabels(ax_ticks)

    # Label the axes
    ax.set_xlabel("Epithelial Axis")
    ax.set_ylabel("Crypt-Villi Axis")

    ax.set_ylim(-0.02, 1.05)
    ax.grid(False)
    # Add a title
    ax.set_title(f"Day 90 Mouse {ucell_key.split('_')[1]} Signature")
    draw_gates(ax, gates=gates, transformation=transformation)

    fig.tight_layout()

    plt.xlim(-19, 8)
    plt.show()
    plt.close()


plot_imaps_signature(adata, gates=gates, ucell_key="UCell_Crypt")
plot_imaps_signature(adata, gates=gates, ucell_key="UCell_Top")